In [ ]:
import telegram
from telegram.ext import Updater, CommandHandler, MessageHandler, Filters
import requests
import time
import base64
import io
from PIL import Image

# Configuration
TESTING_MODE = True
APITOKEN = 'dBPwLbPRc4Gk0dLk9mCRG8QqsLnOtfI9X6xXcsw1b4+wLNtpLV8qZ+sIsVls0+nuIajBG/WPd7k='
IMAGE_FILE = 'photo.jpg'  # Local file to save the downloaded image
API_KEY = "7325643460:AAEB4x9H8P2xBRouV8-V0gt5yCDZC4h5cYM"

# Function to search by face
def search_by_face(image_file):
    if TESTING_MODE:
        print('****** TESTING MODE: Results may be inaccurate, and queue wait is long. Credits are NOT deducted. ******')

    site = 'https://facecheck.id'
    headers = {'accept': 'application/json', 'Authorization': APITOKEN}
    files = {'images': open(image_file, 'rb'), 'id_search': None}
    response = requests.post(f'{site}/api/upload_pic', headers=headers, files=files).json()

    if response['error']:
        return f"Error: {response['error']} ({response['code']})", None

    id_search = response['id_search']
    print(f"{response['message']} id_search={id_search}")
    json_data = {'id_search': id_search, 'with_progress': True, 'status_only': False, 'demo': TESTING_MODE}

    while True:
        response = requests.post(f'{site}/api/search', headers=headers, json=json_data).json()
        if response['error']:
            return f"Error: {response['error']} ({response['code']})", None
        if response['output']:
            return None, response['output']['items']
        print(f"{response['message']} Progress: {response['progress']}%")
        time.sleep(1)

# Command handler for starting the bot
def start(update, context):
    update.message.reply_text("Hello! Hrithik. Send me a photo to check.")
    
def help_bot(update,context):
    update.message.reply_text("""
    Hi there! If you want to access the facecheck.id bot follow these commands:-
    
    /start - to start the bot
    /content - Information about this bot
    /contact - Information about the creater of this bot
    /help - to get help menu
    
    I hope these commands will help you!
    """)
    
def content_bot(update,context):
    update.message.reply_text("""
    Welcome to the Face Search Bot!

    This bot allows you to search the internet for images that match the face in a photo you provide. Here's how to use the bot:

    1. /start: Initiates the bot and gives a welcome message.
    2. /content: Displays this information message.
    3. Send a photo: Simply send a photo to the bot, and it will search the internet for similar images. You will receive:
       - The similarity score
       - The URL of the most matching image
       - A thumbnail of the most related image

    Example Usage:
    - Start by typing /start to get a welcome message.
    - Send a photo of a face.
    - Receive the search results including the similarity score, the URL of the most matching image, and a thumbnail of the most related image.

    Note: For more accurate results, please provide photos that only show the face.

    Enjoy using the Face Search Bot!    
    """)

def contact_bot(update,context):
    update.message.reply_text("""Hello! This is Hrithik, the creator of this bot.
    You can reach out to me via email at goyalhrithik548@gmail.com.""")
    
def casual_text(update,context):
    update.message.reply_text("Please follow the commands below to interact with the bot.")
    help_bot(update,context)

# Message handler for receiving photos
def photo(update, context):
    # Get the photo file
    photo_file = context.bot.getFile(update.message.photo[-1].file_id)
    
    # Download the photo
    photo_file.download(IMAGE_FILE)
    
    # Search the internet by face
    error, urls_images = search_by_face(IMAGE_FILE)
    
    # Reply with the search results or error
    if urls_images:
        score = urls_images[0]['score']
        url = urls_images[0]['url']
        thumbnail_base64 = urls_images[0]['base64']
        
        try:
            # Verify base64 string length
            if len(thumbnail_base64) == 0:
                raise ValueError("Received empty base64 string")

            # Log a part of the base64 string for debugging
            print(f"Base64 thumbnail (first 100 chars): {thumbnail_base64[:100]}")

            # Remove data URL prefix if it exists
            if thumbnail_base64.startswith('data:image/'):
                thumbnail_base64 = thumbnail_base64.split(',')[1]

            # Decode the base64 image
            thumbnail_bytes = base64.b64decode(thumbnail_base64)
            thumbnail_file = io.BytesIO(thumbnail_bytes)
            thumbnail_file.name = 'thumbnail.jpg'

            # Check if the image format can be identified
            try:
                image = Image.open(thumbnail_file)
                print(f"Image format identified: {image.format}")
                image.verify()  # This will raise an exception if the image is not valid
            except Exception as e:
                raise ValueError(f"Image format not recognized: {str(e)}")

            # Seek back to the start of the BytesIO object after verifying
            thumbnail_file.seek(0)

            update.message.reply_photo(photo=thumbnail_file, caption=f"Score: {score}, URL: {url}")
        except Exception as e:
            update.message.reply_text(f"Failed to process the thumbnail image: {str(e)}")
            print(f"Error processing thumbnail image: {str(e)}")
    else:
        update.message.reply_text(error)

def main():
    # Create the Updater and pass it your bot's token
    updater = Updater(API_KEY, use_context=True)
    
    # Get the dispatcher to register handlers
    dp = updater.dispatcher
    
    # Add command handler
    dp.add_handler(CommandHandler("start", start))
    dp.add_handler(CommandHandler("help",help_bot))
    dp.add_handler(CommandHandler("content",content_bot))
    dp.add_handler(CommandHandler("contact",contact_bot))
    dp.add_handler(telegram.ext.MessageHandler(telegram.ext.Filters.text,casual_text))
    
    # Add message handler for receiving photos
    dp.add_handler(MessageHandler(Filters.photo, photo))
    
    # Start the Bot
    updater.start_polling()
    
    # Run the bot until you press Ctrl-C
    updater.idle()

if __name__ == '__main__':
    main()


****** TESTING MODE: Results may be inaccurate, and queue wait is long. Credits are NOT deducted. ******
1 file uploaded. id_search=Ao2RH5ga26s
Waiting in queue. 3rd place Progress: 0%
Waiting in queue. 2nd place Progress: 0%
Waiting in queue. 2nd place Progress: 0%
Waiting in queue. 2nd place Progress: 0%
Waiting in queue. 2nd place Progress: 0%
Waiting in queue. 2nd place Progress: 0%
Waiting in queue. 1st place Progress: 0%
Waiting in queue. 1st place Progress: 0%
Waiting in queue. 1st place Progress: 0%
Waiting in queue. 1st place Progress: 0%
Waiting in queue. 1st place Progress: 0%
Waiting in queue. 1st place Progress: 0%
Base64 thumbnail (first 100 chars): data:image/webp;base64, UklGRgQSAABXRUJQVlA4IPgRAADwYgCdASoEAQQBPqlOoU0mJCOipRKKyMAVCWduwlABafAuTS1O
Image format identified: WEBP
****** TESTING MODE: Results may be inaccurate, and queue wait is long. Credits are NOT deducted. ******
1 file uploaded. id_search=7YAa1LBW9Fw
Waiting in queue. 2nd place Progress: 0%
Waiting in q